<a href="https://colab.research.google.com/github/arkajyotibhattacharya/fundusAI/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FundusAI — Ocular Disease Classification

Automated diagnosis of eye diseases from color fundus photographs using the [ODIR-5K dataset](https://www.kaggle.com/datasets/andrewmvd/ocular-disease-recognition-odir5k/data) (5,000 patients, 8 diagnostic labels).

All logic lives in modular `src/` files — this notebook just runs the pipeline.

In [ ]:
!pip install -q pandas numpy matplotlib seaborn tensorflow scikit-learn kaggle

from warnings import filterwarnings
filterwarnings('ignore')

import numpy as np
import tensorflow as tf
from src.config import *
from src import data_loading as dl, eda, preprocessing as pp, models, training, evaluation as ev

ModuleNotFoundError: No module named 'src.config'

## 1. Load Data
Load the ODIR-5K dataset. Just pass the path to where `full_df.csv` and `ODIR-5K/` live.

In [ ]:
data_ocu = dl.load_odir('ocular_data')
data_ocu.head()

## 2. Explore
Visualize sample fundus images per disease, check class balance, and inspect patient demographics.

In [ ]:
eda.plot_sample_images(data_ocu)
eda.plot_label_distribution(data_ocu)
eda.plot_age_distribution(data_ocu)

## 3. Preprocess
Stratified 60/20/20 train/val/test split, then create image generators that rescale pixel values to [0, 1].

In [ ]:
train_data, val_data, test_data = pp.split_data(data_ocu)
train_gen, val_gen, test_gen = pp.create_generators(train_data, val_data, test_data)

## 4. Baseline CNN
A simple 2-layer convolutional network to establish a performance floor. See `src/config.py` for hyperparameters.

In [ ]:
unique_labels = data_ocu[TARGET_COL].unique()
cnn_model = models.build_baseline_cnn(num_classes=len(unique_labels))
cnn_history, cnn_time = training.train_model(cnn_model, train_gen, val_gen)

## 5. Evaluate
Training/validation curves and test set metrics (accuracy, confusion matrix, classification report).

In [ ]:
ev.plot_training_history(cnn_history)
ev.evaluate_model(cnn_model, test_gen)